In [2]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_2587/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [1]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.discard_below_average(sort_by="r2")
        self.discard_below_average(sort_by="r2_vt")
        self.discard_high_standard_deviation()
        self.results_df.to_excel(f"better_results.xlsx", index=True)
        display(self.results_df)


In [8]:
analize = Analizer(0.9)
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_sup,r2_test,r2_val,r2_vt,mse,mse_sup,mse_test,mse_val,mse_vt,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_6_9_6,0.999605,0.753346,0.999305,0.999051,0.999190,0.000197,0.146425,0.000443,0.000514,0.000478,0.004154,0.014019,1.000056,0.014083,3535.069454,9488.396269,"Hidden Size=[40, 39], regularizer=0.2, learnin..."
1,model_6_9_5,0.999604,0.753542,0.999357,0.999130,0.999254,0.000197,0.146308,0.000410,0.000470,0.000440,0.004219,0.014039,1.000056,0.014104,3535.063636,9488.390452,"Hidden Size=[40, 39], regularizer=0.2, learnin..."
2,model_6_9_7,0.999603,0.753191,0.999257,0.998975,0.999129,0.000198,0.146516,0.000474,0.000555,0.000514,0.004094,0.014056,1.000056,0.014121,3535.058895,9488.385710,"Hidden Size=[40, 39], regularizer=0.2, learnin..."
3,model_6_9_8,0.999598,0.753035,0.999213,0.998901,0.999071,0.000200,0.146609,0.000502,0.000594,0.000548,0.004038,0.014140,1.000057,0.014205,3535.035106,9488.361922,"Hidden Size=[40, 39], regularizer=0.2, learnin..."
4,model_6_9_4,0.999598,0.753708,0.999412,0.999210,0.999321,0.000200,0.146210,0.000375,0.000427,0.000401,0.004289,0.014148,1.000057,0.014213,3535.032823,9488.359639,"Hidden Size=[40, 39], regularizer=0.2, learnin..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
530,model_7_7_17,0.998077,0.796432,0.998454,0.998008,0.998184,0.000956,0.120846,0.000581,0.001166,0.000873,0.016440,0.030924,1.000271,0.031067,3531.904901,9485.231717,"Hidden Size=[40, 39], regularizer=0.5, learnin..."
534,model_35_8_17,0.998061,0.782725,0.998595,0.998185,0.998412,0.000964,0.128984,0.000751,0.000814,0.000782,0.011780,0.031053,1.000681,0.031197,1683.888210,4509.941587,"Hidden Size=[33, 21], regularizer=0.0, learnin..."
537,model_7_7_16,0.998048,0.796432,0.998452,0.998003,0.998180,0.000971,0.120847,0.000581,0.001168,0.000875,0.016573,0.031155,1.000275,0.031299,3531.875071,9485.201886,"Hidden Size=[40, 39], regularizer=0.5, learnin..."
539,model_35_8_16,0.998039,0.782676,0.998608,0.998225,0.998437,0.000975,0.129013,0.000743,0.000796,0.000770,0.011839,0.031228,1.000688,0.031372,1683.865785,4509.919162,"Hidden Size=[33, 21], regularizer=0.0, learnin..."
